In [ ]:
'''
  Install packages required to connect to MySQL
  instance deployed in GCP. The connection is via
  Cloud SQL service offered by GCP
  NOTE: A session restart is required after installation
  of the cloud-sql-python-connector module
'''
# Install PyMySQL python module
!pip install cloud-sql-python-connector

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.33.0 which is incompatible.


In [ ]:
'''
  Install the python MySQL module used for
  managing connections to MySQL DB from
  a python runtime
'''
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.3 MB/s eta 0:00:00


In [ ]:
'''
  Configure the environment with the GCP
  project to which the MySQL instance is
  assigned.
'''
from tools import load_db_cfg
dbcfg = load_db_cfg('pp.json')
#project_id="youtubechannelanalysis-423709"
!gcloud config set project {dbcfg['project_id']}

Updated property [core/project].


In [ ]:
'''
  Aunthenticate to Google Colab as a valid
  GCP user that has access to the above
  project and its resources.
  NOTE: The project needs to be pre-configured
  for the user access
'''
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd
from dbconnect import  DbConnector
import sqlalchemy

class PhonePeDao:
  def __init__(self,dbcfg):
    self.__connector__ = DbConnector(dbcfg)
    self.__connection__ = self.__connector__.connect()

  def get_dashboard_mock(self,rec_type, year, quarter, state):
    if rec_type=='Transaction':
      return  {
              'recordType': 'Transaction',
              'title': 'Transaction Dashboard',
              'description':'Displays transaction information for the selected year or quarter',
              'aggregates':{'name':'Transaction Summary','charts':[{'title': 'Transaction Count', 'value': 50000},{'title': 'Amount (in Rs.)', 'value': 78500}]},
              'top':{'name':'Transactions By Top Locations','charts':[ {'title': 'Top States', 'data': pd.DataFrame({'State':['tamil-nadu','maharastra', 'kerala','karnataka', 'bihar','manipur','andhra-preadesh','jharkand','jammu-&-kashmir', 'puduchery'], 'Transaction Count':[10,9,8,7,6,5,4,3,2,1], 'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})},
                      {'title': 'Top Districts', 'data': pd.DataFrame({'District':['chengalpet','thiruvallur', 'krishnagiri','ramanathapuram','salem','namakkal','karur','coimbatore','nilgiris','hosur'], 'Transaction Count':[10,9,8,7,6,5,4,3,2,1],'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})},
                      {'title': 'Top Pincodes', 'data': pd.DataFrame({'Pincode':['600023','600001', '600011','600028','600038','600048','600058','600068','600078','600088'], 'Transaction Count':[10,9,8,7,6,5,4,3,2,1], 'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})}
              ]},
              'trend':{'name':'Quarterly Transaction Trends', 'charts':[{'title': 'Quarterly Transaction Trend', 'data': pd.DataFrame({'Quarter':['Q1','Q2','Q3','Q4'], 'Transaction Count':[1,4,3,2]})}]},
              'aux_aggregates':{'name': 'Payment Methods','charts':[{'title':'Recharge & bill payments', 'value': 50000,'metric':'Count'},{'title': 'Peer-to-peer payments', 'value': 78500, 'metric':'Count'}, {'title': 'Merchant payments', 'value': 98500,'metric':'Count'},{'title': 'Financial Services', 'value': 50255,'metric':'Count'},{'title':'Recharge & bill payments', 'value': 50000,'metric':'Amount'},{'title': 'Peer-to-peer payments', 'value': 78500, 'metric':'Amount'}, {'title': 'Merchant payments', 'value': 98500,'metric':'Amount'},{'title': 'Financial Services', 'value': 50255,'metric':'Amount'}]}
            }
    if rec_type=='Insurance':
      return  {
                'recordType': 'Insurance',
                'title': 'Insurance Dashboard',
                'description':'Displays insurance information for a given year or quarter',
                'aggregates':{'name':'Insurance Summary', 'charts':[{'title': 'Insurance Count', 'value': 50000},{'title': 'Insurance Amount (in Rs.)', 'value': 78500}]},
                'top':{'name':'Insurance By Top Locations','charts': [{'title': 'Top States', 'data': pd.DataFrame({'State':['tamil-nadu','maharastra', 'kerala','karnataka', 'bihar','manipur','andhra-preadesh','jharkand','jammu-&-kashmir', 'puduchery'], 'Insurance Count':[10,9,8,7,6,5,4,3,2,1], 'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})},
                        {'title': 'Top Districts', 'data': pd.DataFrame({'District':['chengalpet','thiruvallur', 'krishnagiri','ramanathapuram','salem','namakkal','karur','coimbatore','nilgiris','hosur'], 'Insurance Count':[10,9,8,7,6,5,4,3,2,1], 'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})},
                        {'title': 'Top Pincodes', 'data': pd.DataFrame({'Pincode':['600023','600001', '600011','600028','600038','600048','600058','600068','600078','600088'], 'Insurance Count':[10,9,8,7,6,5,4,3,2,1], 'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})}
                  ]},
                  'trend':{'name':'Quarterly Insurance Trends','charts':[{'title': 'Quarterly Insurance Trend', 'data': pd.DataFrame({'Quarter':['Q1','Q2','Q3','Q4'], 'Insurance Count':[1,4,3,2]})}]},
      }
    if rec_type == 'User':
      return {
                'recordType': 'User',
                'title': 'User & Device Dashboard',
                'description':'Displays user and device information for the selected year or quarter',
                'aggregates':{'name':'Registered User Summary', 'charts':[{'title': 'Registered Users', 'value': 50000},{'title': 'Application Opens', 'value': 78500}]},
                'top':{'name':'Registered Users By Top Locations','charts':[ {'title': 'Top States', 'data': pd.DataFrame({'State':['tamil-nadu','maharastra', 'kerala','karnataka', 'bihar','manipur','andhra-preadesh','jharkand','jammu-&-kashmir', 'puduchery'], 'User Count':[10,9,8,7,6,5,4,3,2,1]})},
                        {'title': 'Top Districts', 'data': pd.DataFrame({'District':['chengalpet','thiruvallur', 'krishnagiri','ramanathapuram','salem','namakkal','karur','coimbatore','nilgiris','hosur'], 'Transaction Count':[10,9,8,7,6,5,4,3,2,1]})},
                        {'title': 'Top Pincodes', 'data': pd.DataFrame({'Pincode':['600023','600001', '600011','600028','600038','600048','600058','600068','600078','600088'], 'Transaction Count':[10,9,8,7,6,5,4,3,2,1]})}
                ]},

                'aux_aggregates':{'name':'Registered Devices','charts':[{'title':'Xiaomi', 'value': 11926334},{'title': 'Samsung', 'value': 9609401}, {'title': 'Vivo', 'value': 5894293},{'title': 'Oppo', 'value': 479351},{'title': 'RealMe', 'value': 2376866}, {'title': 'Apple', 'value': 1825153}, {'title': 'Motorolla', 'value': 1593250}, {'title': 'OnePlus', 'value': 1536418}, {'title': 'Lenovo', 'value': 1246507}, {'title': 'Huawei', 'value': 808774}, {'title': 'Others', 'value': 5581520}]},
                'trend':{'name':'Quarterly Trend of Registered Users','charts':[{'title': 'Quarterly User Trend', 'data': pd.DataFrame({'Quarter':['Q1','Q2','Q3','Q4'], 'User Count':[1,4,3,2]})}]}
      }

  def supported_years(self):
    return [2018,2019,2020,2021,2022,2023,2024]

  def supported_quarters(self):
    return [(1,'Q1'),(2,'Q2'),(3,'Q3'),(4,'Q4')]

  def supported_states(self):
    sql= f'''SELECT g.id as 'id', g.name as 'State' FROM geo g WHERE geo_type='STA' ORDER BY name ASC'''

    # Execute agg SQL
    stmt = sqlalchemy.text(sql,)
    df=pd.read_sql_query(stmt, self.__connection__)
    return df


  def get_transaction_dashboard(self,year, quarter=None, state=None):

    # Prepare the dashboard dictionary
    dashboard=dict()
    dashboard['recordType']='Transaction'
    dashboard['title']='Transaction Dashboard'
    dashboard['description']='Displays transaction information for the selected year or quarter'

    # Aggregates Processing
    dashboard['aggregates']=dict()
    agg=dashboard['aggregates']
    agg['name']='Transaction Summary'
    agg['charts']=list()

    # Aux Aggregates
    dashboard['aux_aggregates']=dict()
    aux_agg=dashboard['aux_aggregates']
    aux_agg['name']='Payment Methods'
    aux_agg['charts']=list()

    # SQLs and Filter Procesing
    sql= f'''SELECT t.category as 'Payment Method', sum(t.count) as 'Count', sum(t.amount) 'Amount' FROM transaction_agg t WHERE year=:year'''

    # Set the WHERE clause based on the fiters passed to this function
    params=dict()
    params['year']=year

    if quarter is not None:
      sql += ' AND quarter=:quarter'
      params['quarter']=quarter[0]

    sql += ' AND geo_id=:geo_id'
    if state is not None:
      params['geo_id']=state
    else:
      params['geo_id']=0 #Refers to country India

    sql += ' GROUP BY t.category'

    # Execute agg SQL
    stmt = sqlalchemy.text(sql,)
    df=pd.read_sql_query(stmt, self.__connection__,params=params)

    # Populate 'aggregates' & 'aux_aggregates'
    d_ct_total=dict()
    d_ct_total['title']='Transaction Count'
    d_ct_total['value']=0

    d_amt_total=dict()
    d_amt_total['title']='Amount (in Rs.)'
    d_amt_total['value'] = 0
    for idx, row in df.iterrows():
      d_ct=dict()
      d_ct['title']=row['Payment Method']
      d_ct['value']=row['Count']
      d_ct['metric']='Count'
      aux_agg['charts'].append(d_ct)
      d_ct_total['value'] += row['Count']


      d_amt=dict()
      d_amt['title']=row['Payment Method']
      d_amt['value']=row['Amount']
      d_amt['metric']='Amount'
      aux_agg['charts'].append(d_amt)
      d_amt_total['value'] += row['Amount']

    agg['charts'].append(d_ct_total)
    agg['charts'].append(d_amt_total)


    # Execute top SQL
    if( quarter is not None):
      dashboard['top']=dict()
      sql_top=f'''SELECT g.name as 'Location', g.geo_type as 'Location Type', t.count as 'Transaction Count' ,t.amount as 'Amount (in Rs.)' FROM transaction_top t JOIN geo g ON (t.geo_id=g.id) WHERE year=:year AND quarter=:quarter AND top_in=:top_in'''
      params_top=dict()
      params_top['year']=year
      params_top['quarter']=quarter[0]

      if state is not None:
        params_top['top_in']='STA'
        sql_top += ' AND geo_id IN (SELECT id from geo WHERE parent_id=:parent_id)'
        params_top['parent_id']=state
      else:
        params_top['top_in']='CON'

      stmt = sqlalchemy.text(sql_top,)
      df_top=pd.read_sql_query(stmt, self.__connection__,params=params_top)

      # Populate 'top'
      top=dashboard['top']
      top['name']= 'Transactions By Top Locations'
      top['charts']=list()

      #if state is None:
      top['charts'].append(dict())
      top['charts'][0]['title']='Top States'
      top['charts'][0]['data']=df_top[df_top['Location Type']=='STA'].reset_index().drop(columns=['index']).sort_values('Transaction Count', ascending=False)

      top['charts'].append(dict())
      #cidx=len(top['charts'])-1
      top['charts'][1]['title']='Top Districts'
      top['charts'][1]['data']=df_top[df_top['Location Type']=='DIS'].reset_index().drop(columns=['index']).sort_values('Transaction Count', ascending=False)

      top['charts'].append(dict())
      #cidx=len(top['charts'])-1
      top['charts'][2]['title']='Top Pincodes'
      top['charts'][2]['data']=df_top[df_top['Location Type']=='PIN'].reset_index().drop(columns=['index']).sort_values('Transaction Count', ascending=False)

    # Map Data
    if state is None:
      dashboard['by_state']=dict()
      by_states = dashboard['by_state']
      by_states['name']='Insurance By State'

      params_states=dict()
      params_states['year']=year
      sql_states = f'''SELECT S1.geo_id AS 'State Id', g.name AS 'State', S1.counts AS 'Transaction Count', S1.amount AS 'Transaction Amount' FROM (SELECT geo_id, sum(t.count) as counts, sum(t.amount) as amount FROM transaction_agg t WHERE year=:year'''
      if quarter is not None:
        sql_states += ' AND quarter=:quarter'
        params_states['quarter']=quarter[0]

      sql_states += ''' GROUP BY geo_id)S1 JOIN geo g ON (S1.geo_id=g.id) WHERE geo_type='STA' '''

      stmt = sqlalchemy.text(sql_states,)
      df_states=pd.read_sql_query(stmt, self.__connection__,params=params_states)

      by_states['charts']=list()
      by_states['charts'].append({'title':'Transaction By State','data':df_states})
    return dashboard

  def get_insurance_dashboard(self, year, quarter=None, state=None):

    # Prepare the dashboard dictionary
    dashboard=dict()
    dashboard['recordType']='Insurance'
    dashboard['title']='Insurance Dashboard'
    dashboard['description']='Displays insurance information for the selected year and/or quarter'

    # Aggregates Processing
    dashboard['aggregates']=dict()
    agg=dashboard['aggregates']
    agg['name']='Insurance Summary'
    agg['charts']=list()

    # SQLs and Filter Procesing
    sql= f'''SELECT i.count as 'Count', i.amount as 'Amount' FROM insurance_agg i WHERE year=:year'''

    # Set the WHERE clause based on the fiters passed to this function
    params=dict()
    params['year']=year

    if quarter is not None:
      sql += ' AND quarter=:quarter'
      params['quarter']=quarter[0]

    sql += ' AND geo_id=:geo_id'
    if state is not None:
      params['geo_id']=state
    else:
      params['geo_id']=0 #Refers to country India

    # Execute agg SQL
    stmt = sqlalchemy.text(sql,)
    df=pd.read_sql_query(stmt, self.__connection__,params=params)

    # Populate 'aggregates' & 'aux_aggregates'
    d_ct_total=dict()
    d_ct_total['title']='Insurance Count'
    d_ct_total['value']=0

    d_amt_total=dict()
    d_amt_total['title']='Amount (in Rs.)'
    d_amt_total['value']=0

    for idx, row in df.iterrows():
      d_ct_total['value'] += row['Count']
      d_amt_total['value'] += row['Amount']

    agg['charts'].append(d_ct_total)
    agg['charts'].append(d_amt_total)


    # Execute top SQL
    if quarter is not None:
      dashboard['top']=dict()
      sql_top=f'''SELECT g.name as 'Location', g.geo_type as 'Location Type', i.count as 'Insurance Count', i.amount as 'Amount (in Rs.)' FROM insurance_top i JOIN geo g ON (i.geo_id=g.id) WHERE year=:year AND quarter=:quarter AND top_in=:top_in'''

      params_top=dict()
      params_top['year']=year
      params_top['quarter']=quarter[0]

      if state is not None:
        sql_top += ' AND geo_id IN (SELECT id FROM geo WHERE parent_id=:parent_id)'
        params_top['top_in']='STA'
        params_top['parent_id']=state
      else:
        params_top['top_in']='CON'

      stmt = sqlalchemy.text(sql_top,)
      df_top=pd.read_sql_query(stmt, self.__connection__,params=params_top)

      # Populate 'top'
      top=dashboard['top']
      top['name']= 'Insurance By Top Locations'
      top['charts']=list()

      #if state is None:
      top['charts'].append(dict())
      top['charts'][0]['title']='Top States'
      top['charts'][0]['data']=df_top[df_top['Location Type']=='STA'].reset_index().drop(columns=['index']).sort_values('Insurance Count', ascending=False)

      top['charts'].append(dict())
      #cidx= len(top['charts'])-1
      top['charts'][1]['title']='Top Districts'
      top['charts'][1]['data']=df_top[df_top['Location Type']=='DIS'].reset_index().drop(columns=['index']).sort_values('Insurance Count', ascending=False)

      top['charts'].append(dict())
      #cidx= len(top['charts'])-1
      top['charts'][2]['title']='Top Pincodes'
      top['charts'][2]['data']=df_top[df_top['Location Type']=='PIN'].reset_index().drop(columns=['index']).sort_values('Insurance Count', ascending=False)

    # Map Data
    if state is None:
      dashboard['by_state']=dict()
      by_states = dashboard['by_state']
      by_states['name']='Insurance By State'

      params_states=dict()
      params_states['year']=year
      sql_states = f'''SELECT S1.geo_id AS 'State Id', g.name AS 'State', S1.counts AS 'Insurance Count', S1.amount AS 'Insurance Amount' FROM (SELECT geo_id, sum(i.count) as counts, sum(i.amount) as amount FROM insurance_agg i WHERE year=:year'''
      if quarter is not None:
        sql_states += ' AND quarter=:quarter'
        params_states['quarter']=quarter[0]

      sql_states += ''' GROUP BY geo_id)S1 JOIN geo g ON (S1.geo_id=g.id) WHERE geo_type='STA' '''

      stmt = sqlalchemy.text(sql_states,)
      df_states=pd.read_sql_query(stmt, self.__connection__,params=params_states)

      by_states['charts']=list()
      by_states['charts'].append({'title':'Insurance By State','data':df_states})
    return dashboard

  def get_user_dashboard(self,year, quarter=None, state=None):
    # Prepare the dashboard dictionary
    dashboard=dict()
    dashboard['recordType']='User'
    dashboard['title']='User & Device Dashboard'
    dashboard['description']='Displays user and device information for the selected year or quarter'

    # Aggregates Processing
    dashboard['aggregates']=dict()
    agg=dashboard['aggregates']
    agg['name']='Registered User Summary'
    agg['charts']=list()
    dashboard['aux_aggregates']=dict()
    aux_agg=dashboard['aux_aggregates']
    aux_agg['name']='Registered Devices'
    aux_agg['charts']=list()

    # SQLs and Filter Procesing
    sql= f'''SELECT u.reg_users as 'Registered Users', u.app_opens as 'Application Opens' FROM user_agg u WHERE year=:year'''
    sql_device= f'''SELECT d.brand as 'Brand', sum(d.count) as 'Count', sum(d.percentage) as 'Percentage' FROM device_agg d WHERE year=:year'''

    # Set the WHERE clause based on the fiters passed to this function
    params=dict()
    params['year']=year

    if quarter is not None:
      sql += ' AND quarter=:quarter'
      params['quarter']=quarter[0]

    sql += ' AND geo_id=:geo_id'
    if state is not None:
      params['geo_id']=state
    else:
      params['geo_id']=0 #Refers to country India

    sql_device += ' GROUP BY brand'

    # Execute agg SQL
    stmt = sqlalchemy.text(sql,)
    df=pd.read_sql_query(stmt, self.__connection__,params=params)

    # Populate 'aggregates' & 'aux_aggregates'
    d_reg_users_total=dict()
    d_reg_users_total['title']='Registered Users'
    d_reg_users_total['value']=0

    d_app_opens_total=dict()
    d_app_opens_total['title']='Application Opens'
    d_app_opens_total['value']=0

    for idx, row in df.iterrows():
      d_reg_users_total['value'] += row['Registered Users']
      d_app_opens_total['value'] += row['Application Opens']

    agg['charts'].append(d_reg_users_total)
    agg['charts'].append(d_app_opens_total)

    # Execute device agg SQL
    stmt = sqlalchemy.text(sql_device,)
    df_dagg=pd.read_sql_query(stmt, self.__connection__,params=params)

    for idx, row in df_dagg.iterrows():
      d_ct=dict()
      d_ct['title']=row['Brand']
      d_ct['value']=row['Count']
      d_ct['metric']='Count'
      aux_agg['charts'].append(d_ct)

      d_pct=dict()
      d_pct['title']=row['Brand']
      d_pct['value']=row['Percentage']
      d_pct['metric']='Percentage'
      aux_agg['charts'].append(d_pct)

    # Execute top SQL
    if quarter is not None:
      dashboard['top']=dict()
      sql_top=f'''SELECT g.name as 'Location', g.geo_type as 'Location Type', u.reg_users as 'Registered Users' FROM user_top u JOIN geo g ON (u.geo_id=g.id) WHERE year=:year AND quarter=:quarter AND top_in=:top_in'''

      params_top=dict()
      params_top['year']=year
      params_top['quarter']=quarter[0]

      if state is not None:
        sql_top += ' AND geo_id IN (SELECT id from geo WHERE parent_id=:parent_id)'
        params_top['top_in']='STA'
        params_top['parent_id']=state
      else:
        params_top['top_in']='CON'

      stmt = sqlalchemy.text(sql_top,)
      df_top=pd.read_sql_query(stmt, self.__connection__,params=params_top)

      # Populate 'top'
      top=dashboard['top']
      top['name']= 'Registered Users By Top Locations'
      top['charts']=list()

      #if state is None:
      top['charts'].append(dict())
      top['charts'][0]['title']='Top States'
      top['charts'][0]['data']=df_top[df_top['Location Type']=='STA'].reset_index().drop(columns=['index']).sort_values('Registered Users', ascending=False)

      top['charts'].append(dict())
      #cidx= len(top['charts'])-1
      top['charts'][1]['title']='Top Districts'
      top['charts'][1]['data']=df_top[df_top['Location Type']=='DIS'].reset_index().drop(columns=['index']).sort_values('Registered Users', ascending=False)

      top['charts'].append(dict())
      #cidx= len(top['charts'])-1
      top['charts'][2]['title']='Top Pincodes'
      top['charts'][2]['data']=df_top[df_top['Location Type']=='PIN'].reset_index().drop(columns=['index']).sort_values('Registered Users', ascending=False)

    # Map Data
    if state is None:
      dashboard['by_state']=dict()
      by_states = dashboard['by_state']
      by_states['name']='User Statistics By State'

      params_states=dict()
      params_states['year']=year
      sql_states = f'''SELECT S1.geo_id AS 'State Id', g.name AS 'State', S1.rusers AS 'Registered Users', S1.aopens AS 'App Opens'  FROM (SELECT geo_id, sum(u.reg_users) as rusers, sum(u.app_opens) as aopens FROM user_agg u WHERE year=:year'''
      if quarter is not None:
        sql_states += ' AND quarter=:quarter'
        params_states['quarter']=quarter[0]

      sql_states += ''' GROUP BY geo_id)S1 JOIN geo g ON (S1.geo_id=g.id) WHERE geo_type='STA' '''

      stmt = sqlalchemy.text(sql_states,)
      df_states=pd.read_sql_query(stmt, self.__connection__,params=params_states)

      by_states['charts']=list()
      by_states['charts'].append({'title':'User Statistics By State','data':df_states})


    return dashboard

  def get_dashboard(self,rec_type,year,quarter=None,state=None):
   if rec_type == 'Transaction':
    return self.get_transaction_dashboard(year,quarter,state)
   elif rec_type=='Insurance':
    return self.get_insurance_dashboard(year, quarter,state)
   else:
    return self.get_user_dashboard(year, quarter, state)

  def close(self):
    self.__connection__.close()








In [ ]:
# Test Transaction Dashboad

# Read DB config
from tools import load_db_cfg
dbcfg = load_db_cfg('pp.json')

# Create DAO
dao=PhonePeDao(dbcfg)

# Quarterly Dashboard
d1 = dao.get_transaction_dashboard(2018,quarter=(1,'Q1'))

# Yearly Dashboard
d2=dao.get_transaction_dashboard(2018)

# State filter
d3 = dao.get_transaction_dashboard(2018,quarter=(1,'Q1'),state=2)

dao.close()
print(f'''
# Quaraterly Dashboard
-----------------------
 {d1}


# Yearly Dashboard
------------------

{d2}

# Quarterly (w/ State Filter)
------------------

{d3}
''')





In [ ]:
# Test Insurance Dashboad

# Read DB config
from tools import load_db_cfg
dbcfg = load_db_cfg('pp.json')

# Create DAO
dao=PhonePeDao(dbcfg)

# Quarterly Dashboard
d1 = dao.get_insurance_dashboard(2024,quarter=(1,'Q1'))

# Yearly Dashboard
d2=dao.get_insurance_dashboard(2024)

# State filter
d3 = dao.get_insurance_dashboard(2024,quarter=(1,'Q1'),state=2)
dao.close()
print(f'''
# Quaraterly Dashboard
-----------------------
 {d1}


# Yearly Dashboard
------------------

{d2}

# Quarterly (w/ State Filter)
------------------

{d3}
''')

In [ ]:
# Test User Dashboad

# Read DB config
from tools import load_db_cfg
dbcfg = load_db_cfg('pp.json')

# Create DAO
dao=PhonePeDao(dbcfg)

# Quarterly Dashboard
d1 = dao.get_user_dashboard(2024,quarter=(1,'Q1'))

# Yearly Dashboard
d2=dao.get_user_dashboard(2024)

# State filter
d3=dao.get_user_dashboard(2024,quarter=(1,'Q1'),state=2)


dao.close()
print(f'''
# Quaraterly Dashboard
-----------------------
 {d1}


# Yearly Dashboard
------------------

{d2}

# Quarterly (w/ State Filter)
------------------

{d3}
''')

Successfully connected to 'phonepe' database!! 

# Quaraterly Dashboard
-----------------------
 {'recordType': 'User', 'title': 'User & Device Dashboard', 'description': 'Displays user and device information for the selected year or quarter', 'aggregates': {'name': 'Registered User Summary', 'charts': [{'title': 'Registered Users', 'value': 530084540}, {'title': 'Registered Users', 'value': 530084540}]}, 'aux_aggregates': {'name': 'Registered Devices', 'charts': [{'title': 'Unknown', 'value': 0.0, 'metric': 'Count'}, {'title': 'Unknown', 'value': 3700.0, 'metric': 'Percentage'}]}, 'top': {'name': 'Registered Users By Top Locations', 'charts': [{'title': 'Top States', 'data':          Location Location Type  Registered Users
4     maharashtra           STA          68016876
8   uttar-pradesh           STA          56028765
2       karnataka           STA          43668141
0  andhra-pradesh           STA          34008709
5       rajasthan           STA          33505514
9     west-beng